So Far imputing missing data with a negative integer leads to worse results compared to ignoring them but leads to faster training time

Testing different imputing methods

Current Best

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [50]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [51]:
train = train.drop('id', axis = 1)
test = test.drop('id', axis = 1)

In [52]:
pd.set_option('display.max_columns', None)

In [53]:
def replace_non_alpha_with_nan(df):
    cols_to_filter = ['cap-shape', 'cap-surface', 'cap-color', 
                      'does-bruise-or-bleed', 'gill-attachment', 
                      'gill-spacing', 'gill-color', 'stem-surface', 
                      'stem-color', 'has-ring', 'ring-type', 'habitat', 'stem-root', 'veil-type', 'veil-color', 'spore-print-color']

    alphabet_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 
                     'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    col_values = {}
    for col in cols_to_filter:
        value_counts = train[col].value_counts()
        col_values[col] = value_counts[value_counts > 10].index.values.tolist()


    def filter_alpha(value, value_list_no_outliers):
        if isinstance(value, str):
            return value if len(value) == 1 and value in value_list_no_outliers and value in alphabet_list else np.nan # if value is a single character
        
        return np.nan
    
    for col in cols_to_filter:
        df[col] = df[col].apply(lambda x : filter_alpha(x, col_values[col]))

    return df

In [54]:
train = replace_non_alpha_with_nan(train)
test = replace_non_alpha_with_nan(test)

In [55]:
cat_cols = [col for col in train.select_dtypes('object').columns if col != 'class']
num_cols = [col for col in train.select_dtypes('number').columns]
print(f'Categorical columns:\n {cat_cols}\n')
print(f'Numeric columns:\n {num_cols}')

Categorical columns:
 ['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']

Numeric columns:
 ['cap-diameter', 'stem-height', 'stem-width']


In [56]:
# train_combinations = train_preprocessed['cat__class'].to_frame()

# for i, col in enumerate(cat_cols.columns):
#     if col != 'cat__class':
#         for j, col2 in enumerate(cat_cols[i+1:], start=i+1):
#             if col2 != 'cat__class':
#                 combination = cat_cols[col] * cat_cols[col2]
#                 train_combinations = train_combinations.join(combination.rename(f'{col} x {col2}'))

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

def handle_missing_data(df, num_cols, cat_cols):

    numeric_transformer = Pipeline(steps = [
        ('imputer', KNNImputer(n_neighbors = 3))
    ])

    categorical_transformer = Pipeline(steps = [
        # ('imputer', SimpleImputer(strategy = 'most_frequent')),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ])

    preprocessor = ColumnTransformer(
        transformers = [
            ('num', numeric_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)
        ]
    )

    df_transformed = pd.DataFrame(preprocessor.fit_transform(df[num_cols + cat_cols]), columns = num_cols + cat_cols)

    df_transformed = df_transformed.fillna(-20)

    print("Missing values after imputation:")
    print(df_transformed.isnull().sum())

    df_final = df[['class']].join(df_transformed)

    return df_final

In [58]:
train = handle_missing_data(train, num_cols, cat_cols)

Missing values after imputation:
cap-diameter            0
stem-height             0
stem-width              0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64


In [59]:
def handle_missing_testset(df, num_cols, cat_cols):

    numeric_transformer = Pipeline(steps = [
        ('imputer', KNNImputer(n_neighbors=3))
    ])

    categorical_transformer = Pipeline(steps = [
        # ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = -1))
    ])

    preprocessor = ColumnTransformer(
        transformers = [
            ('num', numeric_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)
        ]
    )

    df_transformed = pd.DataFrame(preprocessor.fit_transform(df[num_cols + cat_cols]), columns = num_cols + cat_cols)

    df_transformed = df_transformed.fillna(-20)

    print("Missing values after imputation:")
    print(df_transformed.isna().sum())

    df_final = df_transformed
    return df_final

In [60]:
test = handle_missing_testset(test, num_cols, cat_cols)

Missing values after imputation:
cap-diameter            0
stem-height             0
stem-width              0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64


In [61]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['class'] = le.fit_transform(train['class'])

In [62]:
y = train['class']
X = train.drop('class', axis = 1)

In [63]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [64]:
from sklearn.metrics import matthews_corrcoef

def mcc_metric(y_pred, dmatrix):
    y_true = dmatrix.get_label()
    y_pred = (y_pred > 0.5).astype(int) 
    mcc = matthews_corrcoef(y_true, y_pred)
    return 'mcc', mcc

In [65]:
from sklearn.metrics import matthews_corrcoef
from xgboost import XGBClassifier
import gc

model = XGBClassifier(

    colsample_bytree = 0.6,
    max_depth = 14,
    min_child_weight = 7,
    random_state = 42,
    n_estimators = 200,
)

In [66]:
XGB = model.fit(
    X_train, 
    y_train, 
    eval_set = [(X_test, y_test)],
    eval_metric = mcc_metric)

c:\Users\hugph\ML\s4e8\venv\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.45915	validation_0-mcc:0.93866
[1]	validation_0-logloss:0.32880	validation_0-mcc:0.96137
[2]	validation_0-logloss:0.24698	validation_0-mcc:0.96997
[3]	validation_0-logloss:0.19155	validation_0-mcc:0.97272
[4]	validation_0-logloss:0.14739	validation_0-mcc:0.97675
[5]	validation_0-logloss:0.12306	validation_0-mcc:0.97783
[6]	validation_0-logloss:0.10025	validation_0-mcc:0.97940
[7]	validation_0-logloss:0.08378	validation_0-mcc:0.98079
[8]	validation_0-logloss:0.07380	validation_0-mcc:0.98088
[9]	validation_0-logloss:0.06345	validation_0-mcc:0.98155
[10]	validation_0-logloss:0.05943	validation_0-mcc:0.98149
[11]	validation_0-logloss:0.05312	validation_0-mcc:0.98197
[12]	validation_0-logloss:0.04922	validation_0-mcc:0.98229
[13]	validation_0-logloss:0.04612	validation_0-mcc:0.98259
[14]	validation_0-logloss:0.04417	validation_0-mcc:0.98269
[15]	validation_0-logloss:0.04263	validation_0-mcc:0.98279
[16]	validation_0-logloss:0.04137	validation_0-mcc:0.98291
[17]	va

In [67]:
y_pred = XGB.predict(X_test)

In [68]:
score = matthews_corrcoef(y_test, y_pred)
print('MCC', score)

MCC 0.9837487953056541


In [69]:
test_pred_prob = XGB.predict(test)

In [70]:
test_pred_class = le.inverse_transform(test_pred_prob)

In [71]:
submission = pd.read_csv('sample_submission.csv')

submission['class'] = test_pred_class

In [72]:
submission.to_csv('submission.csv', index = False)